In [3]:
import os

import numpy as np
import tvm
from tvm import relay, autotvm
import tvm.relay.testing
from tvm.autotvm.tuner import XGBTuner, GATuner, RandomTuner, GridSearchTuner
from tvm.contrib.utils import tempdir
import tvm.contrib.graph_executor as runtime
import onnx
import glob

In [4]:
op9_dla_model = onnx.load('../op9_dla/20210622_320_192_op9_dla.onnx')

In [5]:
input_name = "input.1"
shape_dict = {input_name: (1, 3, 320, 192)}
print(shape_dict)

{'input.1': (1, 3, 320, 192)}


In [6]:
model, params = relay.frontend.from_onnx(op9_dla_model, shape_dict)

In [7]:
target      = tvm.target.Target("llvm -device=arm_cpu -mtriple=aarch64-linux-gnu -mattr=+neon")
target_host = tvm.target.Target("llvm -device=arm_cpu -mtriple=aarch64-linux-gnu -mattr=+neon")

# Also replace this with the device key in your tracker
device_key = "v9h"

# Set this to True if you use android phone
use_android = False

#### TUNING OPTION ####
network = "op9_cla"
log_file = "%s.%s.log" % (device_key, network)
dtype = "float32"

In [8]:
tuning_option = {
    "log_filename": log_file,
    "tuner": "xgb",
    "n_trial": 1500,
    "early_stopping": 800,
    "measure_option": autotvm.measure_option(
        builder=autotvm.LocalBuilder(build_func="ndk" if use_android else "default"),
        runner=autotvm.RPCRunner(
            device_key,
            host="192.168.105.70",
            port=9190,
            number=5,
            timeout=10,
        ),
    ),
}

In [9]:
tasks = autotvm.task.extract_from_program(model["main"], target=target, params=params, ops=(relay.op.get("nn.conv2d"),))

In [11]:
def tune_tasks(
    tasks,
    measure_option,
    tuner="xgb",
    n_trial=1000,
    early_stopping=None,
    log_filename="tuning.log",
    use_transfer_learning=True,
):
    # create tmp log file
    tmp_log_file = log_filename + ".tmp"
    if os.path.exists(tmp_log_file):
        os.remove(tmp_log_file)

    for i, tsk in enumerate(reversed(tasks)):
        prefix = "[Task %2d/%2d] " % (i + 1, len(tasks))

        # create tuner
        if tuner == "xgb" or tuner == "xgb-rank":
            tuner_obj = XGBTuner(tsk, loss_type="rank")
        elif tuner == "xgb_knob":
            tuner_obj = XGBTuner(tsk, loss_type="rank", feature_type="knob")
        elif tuner == "xgb_itervar":
            tuner_obj = XGBTuner(tsk, loss_type="rank", feature_type="itervar")
        elif tuner == "xgb_curve":
            tuner_obj = XGBTuner(tsk, loss_type="rank", feature_type="curve")
        elif tuner == "ga":
            tuner_obj = GATuner(tsk, pop_size=50)
        elif tuner == "random":
            tuner_obj = RandomTuner(tsk)
        elif tuner == "gridsearch":
            tuner_obj = GridSearchTuner(tsk)
        else:
            raise ValueError("Invalid tuner: " + tuner)

        if use_transfer_learning:
            if os.path.isfile(tmp_log_file):
                tuner_obj.load_history(autotvm.record.load_from_file(tmp_log_file))

        # process tuning
        tsk_trial = min(n_trial, len(tsk.config_space))
        tuner_obj.tune(
            n_trial=tsk_trial,
            early_stopping=early_stopping,
            measure_option=measure_option,
            callbacks=[
                autotvm.callback.progress_bar(tsk_trial, prefix=prefix),
                autotvm.callback.log_to_file(tmp_log_file),
            ],
        )

    # pick best records to a cache file
    autotvm.record.pick_best(tmp_log_file, log_filename)
    os.remove(tmp_log_file)


In [ ]:
print("Tuning...")
tune_tasks(tasks, **tuning_option)

Tuning...
[Task  1/34]  Current/Best:   15.43/  38.56 GFLOPS | Progress: (63/1008) | 210.43 s

/home/zgh/.local/share/virtualenvs/ustc-PR_5tIRD/lib/python3.6/site-packages/xgboost/training.py:17: UserWarning: Old style callback is deprecated.  See: https://xgboost.readthedocs.io/en/latest/python/callbacks.html
  warnings.warn(f'Old style callback is deprecated.  See: {link}', UserWarning)


[Task  1/34]  Current/Best:    4.02/  41.75 GFLOPS | Progress: (876/1008) | 2931.33 s Done.
[Task  2/34]  Current/Best:   12.17/  33.32 GFLOPS | Progress: (315/1500) | 1490.50 s